In [24]:
from pywr.core import Model
from pywr.notebook import draw_graph
import matplotlib.pyplot as plt
import os
from pywr.domains.groundwater import KeatingAquifer
from pywr.parameters.groundwater import KeatingStreamFlowParameter
from pywr.core import Model, Input, Output, Link, Storage, Node, Catchment
from pywr.recorders import (
    NumpyArrayNodeRecorder,
    NumpyArrayLevelRecorder,
    NumpyArrayStorageRecorder,
)
import pandas
import numpy as np
import pytest

def createAquifer(model,name):
    num_streams = 1
    num_additional_inputs = 0
    stream_flow_levels = [[1.0, 1.25]]  # m
    transmissivity = [100, 2000]  # m2/d
    transmissivity = [t * 0.001 for t in transmissivity]  # m3 to Ml
    coefficient = 1  # no units
    storativity = [0.0005]  # % este parámetro es clave
    levels = [1.0, 10.0]  # m
    area = 50000 * 50000  # m2
    
    aqfer=KeatingAquifer(
        model,
        name,
        num_streams,
        num_additional_inputs,
        stream_flow_levels,
        transmissivity,
        coefficient,
        levels,
        area=area,
        storativity=storativity)

    aqfer.initial_level=1
    
    return aqfer

if __name__ == "__main__":
    
    path="nuble.json"
    model = Model.load(path)
    
#     crear objetos acuíferos
    aqferNiquen=createAquifer(model,'AC01_NIQ')
    aqferNuble=createAquifer(model,'AC03_NUB')
    
#     conectar nodos existentes a los acuíferos
    model.nodes['REMZemitaI'].connect(aqferNiquen)
    model.nodes['REMVirguinJFRI'].connect(aqferNiquen)
    aqferNiquen.connect(model.nodes['abstraccionAqNiq'],
from_slot=0)
    aqferNiquen.connect(model.nodes['Zemita_mix'],
                            from_slot=0)
    aqferNiquen.connect(model.nodes['VirguinJFR_mix'],
                            from_slot=0)
    
    model.nodes['REMCollicoI'].connect(aqferNuble)
    aqferNuble.connect(model.nodes['abstraccionAqNub'],
from_slot=0)
    aqferNuble.connect(model.nodes['deficitCollico'],
                            from_slot=0)
    
    model.nodes['deficitCollico'].connect(model.nodes['Collico_mix'])
#     correr modelo
    model.solver.name="glpk-edge"
    stats = model.run()
    print(stats)
    
    plt.figure()
    draw_graph(model,labels=True,attributes=True,
               width=900,height=900)
    plt.show()


Simplex solve returned: None (0)
Simplex status: no feasible solution exists (4)
Scenario ID: 0
Timestep index: 1
Writing problem data to 'pywr_glpk_debug.mps'...
774 records were written
Writing problem data to 'pywr_glpk_debug.lp'...
331 lines were written
GLPK Simplex Optimizer 5.0
133 rows, 83 columns, 1051 non-zeros
123: obj =  -6.887748571e+06 inf =   3.074e+01 (2)

LP HAS NO PRIMAL FEASIBLE SOLUTION


RuntimeError: Simplex solver failed with message: "None", status: "no feasible solution exists".

In [23]:
# bocatoma 3
print(1000-36.8-model.nodes["bocatoma3"].flow)
#validar el balance de masa en canal Virguin
print(model.nodes["FRVirguin"].flow)
print(model.nodes["REMVirguinJFR"].flow)
#validar balance de masa en canal Zemita
print(model.nodes["REMZemita"].flow)


[28.57142857]
[79.65714286]
[14.28571429]
[0.]


In [2]:
# from pywr.notebook import to_html
# to_html(m,height=800,width=800,labels=True,attributes=True,title='Modelo Cuenca del Río Huasco')

In [10]:
from pywr.notebook import pywr_model_to_d3_json
d3json=pywr_model_to_d3_json(model)
import json
with open("./d3.json", "w") as fp:
    json.dump(d3json,fp) 